<a href="https://colab.research.google.com/github/haizznaam/Content-Based-and-Tensorflow-Movie-Recommender-System/blob/main/knn_recommendation_system_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.chdir('/content/drive/MyDrive/Movie Recommendation System/The-Movie-Dataset/')
print("We are currently in the folder of ", os.getcwd())

We are currently in the folder of  /content/drive/MyDrive/Movie Recommendation System/The-Movie-Dataset


# [KNN](#KNN)
- ## [Data preperation](##KNN-Data-preperation)
- ## [KNN Implamentation and Training](##KNN-Implamentation-and-Training)
- ## [Implamenting Prediction and Evaluation](##Implamenting-Prediction-and-Evaluation)
- ## [Testing different K values](##Testing-different-K-values)
- ## [Adding new Users or Items or rating and adressing the cold start problem](##Adding-new-Users-or-Items-or-rating-and-adressing-the-cold-start-problem)

# KNN

Understanding KNN:

https://www.youtube.com/watch?v=jw5LhTWUoG4&t=127s

https://www.youtube.com/watch?v=kccT0FVK6OY

https://chat.openai.com/share/837847bb-ae5f-4cc3-a8a3-a664f352365f

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

Prepearing the data for KNN is a bit different then for SVD. We are gonna look at two examples 1. Where we have all possible user entries in our KNN matrix and all of our possible item entries. So in other words we have all the users and movies that are ever going to be used on our algorithem.

In the real world things area bit different since new users and moveis / items are added every day. the 2. metod we will look at is upgrading our algorithem to be able to handle  new users and items respectively.  

## KNN Data preperation

## 1. Static user and item entries no new users
For this aproach we will gaurantee that each user and each item has an entry in our trainign dataset. Because for KNN we need entries to predict unlike SVD. That's because KNN works on other user's rating for similar entries unlike SVD that mathamaticaly models latent factors and can "implicitly" "learn" what similar data might be rated as.

Đối với phương pháp này, chúng tôi sẽ đảm bảo rằng mỗi user và mỗi item đều có một entry trong tập dữ liệu huấn luyện của chúng tôi.

Vì đối với KNN chúng ta cần các entries để dự đoán,  không giống như SVD. Đó là bởi vì KNN hoạt động dựa trên xếp hạng của người dùng khác cho các mục tương tự không giống như SVD mà toán học mô hình hóa các yếu tố tiềm ẩn và có thể "ngầm" "tìm hiểu" dữ liệu tương tự có thể được xếp hạng là gì.

The following is my first intuitive attempt at spliting and prepearing the data, but it is wrong we will get to that shortly:

In [5]:
# Load the dataset
df = pd.read_csv('ratings_small.csv')

# Display the first few rows of the dataframe
df.head()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
df.columns = ['user_id', 'item_id', 'rating', 'timestamp']
df.head()

,user_id,item_id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
# Perform a train-test split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Check the size of the resulting splits
print(f"Train data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")

Train data size: (80003, 4)
Test data size: (20001, 4)


In [8]:
num_users = train_data['user_id'].nunique()
num_items = train_data['item_id'].nunique()
print("number of unique users:", num_users)
print("number of unique items:", num_items)

number of unique users: 671
number of unique items: 8399


In [9]:
# Find the minimum user_id in the train and test datasets
min_user_id_train = train_data['user_id'].min()
max_user_id_train = train_data['user_id'].max()

print(f"Minimum user_id in train dataset: {min_user_id_train}")
print(f"Maximum user_id in train dataset: {max_user_id_train}")

print('=' * 40)
# Find the maximum user_id in the train and test datasets
min_user_id_test = test_data['user_id'].min()
max_user_id_test = test_data['user_id'].max()

print(f"Minimum user_id in test dataset: {min_user_id_test}")
print(f"Maximum user_id in test dataset: {max_user_id_test}")

Minimum user_id in train dataset: 1
Maximum user_id in train dataset: 671
Minimum user_id in test dataset: 1
Maximum user_id in test dataset: 671


In [10]:
# Find the minimum item_id in the train and test datasets
min_item_id_train = train_data['item_id'].min()
max_item_id_train = train_data['item_id'].max()

print(f"Minimum item_id in train dataset: {min_item_id_train}")
print(f"Maximum item_id in train dataset: {max_item_id_train}")

print('=' * 40)
# Find the maximum item_id in the train and test datasets
min_item_id_test = test_data['item_id'].min()
max_item_id_test = test_data['item_id'].max()

print(f"Minimum item_id in test dataset: {min_item_id_test}")
print(f"Maximum item_id in test dataset: {max_item_id_test}")

Minimum item_id in train dataset: 1
Maximum item_id in train dataset: 163949
Minimum item_id in test dataset: 1
Maximum item_id in test dataset: 162672


In [11]:
last_user = test_data['user_id'].max()
last_item = test_data['item_id'].max()
print("last user id:", last_user)
print("last item id:", last_item)

last user id: 671
last item id: 162672


In [12]:
train_data.head()

,user_id,item_id,rating,timestamp
37865,273,5816,4.5,1466946328
46342,339,2028,4.5,1446663181
64614,461,3895,0.5,1093224965
41974,300,3578,4.5,1086010878
50236,369,292,3.0,847465462


In [13]:
# Create a pivot table for the train data to form the user-item matrix
train_data_matrix_0_fill = train_data.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)

# Display the shape and first few rows of the matrix
display(train_data_matrix_0_fill.head())
train_data_matrix_0_fill = train_data_matrix_0_fill.to_numpy()
print(train_data_matrix_0_fill.shape)

item_id,1,2,3,4,5,6,7,8,9,10,...,160656,160718,161084,161155,161594,161830,161918,161944,162542,163949
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0


(671, 8399)


The reason that the above code for prepearing data is wrong is because in the above code I didn't implicitly make sure that our train_data_matrix_0_fill has an entry for each possible user and each possible item in our dataset.
As we can see:

In [14]:
num_users = df['user_id'].nunique()
num_items = df['item_id'].nunique()
print("number of full unique users:", num_users)
print("number of full unique items:", num_items)

number of full unique users: 671
number of full unique items: 9066


In [15]:
num_users = train_data['user_id'].nunique()
num_items = train_data['item_id'].nunique()
print("number of train unique users:", num_users)
print("number of train unique items:", num_items)

number of train unique users: 671
number of train unique items: 8399


As we can see from the above codse in our training data that we split we have 671 users which is ok but we only have 8399 items which is not okey because when we look at the entire dataset(the df dataset) without the split we can see that we have 9066 movies and 8399 != 9066...

Also if we look at the biggest values of user id's and item id's in our test data we quickly see the same problem:

In [16]:
last_user = train_data['user_id'].max()
last_item = train_data['item_id'].max()
print("last user id:", last_user)
print("last item id:", last_item)

print("!!!does not equal to:")
print("number of unique users:", num_users)
print("number of unique items:", num_items)

last user id: 671
last item id: 163949
!!!does not equal to:
number of unique users: 671
number of unique items: 8399


We can again see that in our test data we have a item with the biggest index id of 163949 but we only have 8399 items in our training dataset because the random split didn't make sure all of our possible values are represneted in out training set and that is a problem for KNN unless we update all the values in our Knn matrix which we will explore in a future step. For now let's fix this problem by making sure each user, item id value is represented in our training data:

SO as we just astablished the previous data preperation and split was wrong and so we will rather do it like:

Here is what we will do in the code:

- take the df dataset and duplicate it into df_copy that isn't split yet.

- Then firstly find the first apearence of each unique user id and take the entry in df_copy out of df_copy and add it to training_data.

- Then find the first unique apearence of each item id in df (not df_copy) if it already exists inside of train_data and dosen't inside of df_copy (so it was taken out of df copy allready when we were looking for apearences of user id's ) then don't do anything.

- If on the other hand it isn't inside of train_data and it is inside fo df_copy again take it out of df_copy and put it into train_data.

- After that split the df_copy like we did with df before a 75 / 25 split into training_remaining and test_data. Then simply add the training_remeining onto training_data.

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming df is already loaded
df_copy = df.copy()

# Collect indices for users
user_indices = []
for user_id in df['user_id'].unique():
    user_index = df_copy[df_copy['user_id'] == user_id].index[0]
    user_indices.append(user_index)

# Collect indices for items not already covered by user selection
item_indices = []
for item_id in df['item_id'].unique():
    if not df_copy.loc[df_copy['item_id'] == item_id].index.intersection(user_indices).any():
        item_index = df_copy[df_copy['item_id'] == item_id].index[0]
        item_indices.append(item_index)

# Combine indices and drop duplicates in case of overlap
combined_indices = list(set(user_indices + item_indices))

# Extract rows for training data
train_data_initial = df_copy.loc[combined_indices]

# Drop these rows from df_copy
df_copy = df_copy.drop(index=combined_indices)

# Perform a 75/25 split on the remaining data
training_remaining, test_data = train_test_split(df_copy, test_size=0.25, random_state=42)

# Combine training_remaining with the initial extracted training data
train_data = pd.concat([train_data_initial, training_remaining]).reset_index(drop=True)

# Output the sizes of the final splits
print(f"Train data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")


Train data size: (77411, 4)
Test data size: (22593, 4)


In [18]:
num_users = train_data['user_id'].nunique()
num_items = train_data['item_id'].nunique()
print("number of unique users:", num_users)
print("number of unique items:", num_items)

number of unique users: 671
number of unique items: 9066


In [19]:
last_user = test_data['user_id'].max()
last_item = test_data['item_id'].max()
print("test last user id:", last_user)
print("test last item id:", last_item)
print('=' * 40)
last_user = df['user_id'].max()
last_item = df['item_id'].max()
print("full last user id:", last_user)
print("full last item id:", last_item)

print('=' * 40)
last_user = train_data['user_id'].max()
last_item = train_data['item_id'].max()
print("train last user id:", last_user)
print("train last item id:", last_item)

test last user id: 671
test last item id: 159093
full last user id: 671
full last item id: 163949
train last user id: 671
train last item id: 163949


As we can see the values for df and train are now the same and that is important! The test sets last item id: 159093 is not the same but that is okey because we are not training on the test data so there is no problem in that regard.

In [20]:
# Create a pivot table for the train data to form the user-item matrix
train_data_matrix_0_fill = train_data.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)

# Display the shape and first few rows of the matrix
display(train_data_matrix_0_fill.head())
train_data_matrix_0_fill = train_data_matrix_0_fill.to_numpy()
print(train_data_matrix_0_fill.shape)

item_id,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0


(671, 9066)


We can also do the same but use avrages instead of 0's to fill the missing values. We will use this initialization alter on to see if it makes any difference for now we will just prepare the matrix!

In [21]:
user_rating_avg = train_data.groupby('user_id')['rating'].mean()
user_rating_avg

user_id
1      2.550000
2      3.493333
3      3.590000
4      4.348259
5      3.917526
         ...   
667    3.666667
668    3.785714
669    3.444444
670    3.954545
671    3.925676
Name: rating, Length: 671, dtype: float64

In [22]:
# Create the pivot table
train_data_matrix = train_data.pivot_table(index='user_id', columns='item_id', values='rating')
# Fill missing values with user averages
train_data_matrix_avg_fill = train_data_matrix.apply(lambda x: x.fillna(user_rating_avg[x.name]), axis=1)
# Display the shape and first few rows of the matrix
train_data_matrix_avg_fill.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,...,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000
2,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,4.000000,...,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333
3,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,...,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000
4,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.000000,...,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259
5,3.917526,3.917526,4.000000,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,...,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526


## KNN Implamentation and Training

The following function calculates the similareties between two user entires based on their movie ratings. In this notebook we will mainly focus on the euclidian ecimilarity but later we will experiment with the cosine and paerson similareties

In [23]:
import numpy as np

def calculate_similarity_between_users(user1, user2, similarity_type='euclidean'):
    if similarity_type == 'euclidean':
        # Calculate the Euclidean distance
        distance = np.sqrt(np.nansum((user1 - user2) ** 2))
        # Convert distance to similarity
        similarity = 1 / (1 + distance)
        return similarity
    else:
        raise ValueError("Unsupported similarity type")

Below is the function that trains out KNN matrix. It find the k nearest neighbours based on the similareties between them. It also returns a similareties matrix that we can use for more fine tuned predictions

In [24]:
import time
def k_nearest_neighbors(dataset_matrix, k, similarity_type='euclidean', print_=True, print_every_n=100):
    # Convert the pandas DataFrame to a NumPy array if it's not already an array
    if isinstance(dataset_matrix, pd.DataFrame):
        dataset_array = dataset_matrix.to_numpy()
    else:
        dataset_array = dataset_matrix

    num_users = dataset_array.shape[0]
    # Initialize the KNN matrix with zeros
    knn_matrix = np.zeros((num_users, k), dtype=int)
    similarities_matrix = np.zeros((num_users, k), dtype=float)

    # Placeholder for the similarity scores and user indices
    # We use np.inf as the initial values to ensure any actual similarity score will be lower
    best_similarities = np.full((k, 2), np.inf) # [similarity, index]

    start_total_time = time.time()  # Start timing for the whole function

    for i in range(num_users):
        start_time = time.time()  # Start timing for the current user
        for j in range(num_users):
            if i != j:
                similarity = calculate_similarity_between_users(dataset_array[i], dataset_array[j], similarity_type)
                # Check if this similarity is better than the current stored similarities
                for l in range(k-1, -1, -1):
                    if similarity < best_similarities[l, 0]:
                        if l < k - 1:
                            # Shift the later values down one position
                            best_similarities[l+1] = best_similarities[l]
                        best_similarities[l] = [similarity, j]
                    else:
                        break

        # Store the indices of the K nearest neighbors
        knn_matrix[i] = best_similarities[:, 1].astype(int)
        similarities_matrix[i] = best_similarities[:, 0].astype(float)
        # Reset the best similarities for the next user
        best_similarities.fill(np.inf)

        end_time = time.time()  # End timing for the current user
        if(print_ and i % print_every_n == 0): print(f"Processed user {i+1}/{num_users} in {end_time - start_time:.2f} seconds.")

    end_total_time = time.time()  # End timing for the whole function
    if(print_): print(f"Total time taken: {end_total_time - start_total_time:.2f} seconds.")


    return knn_matrix, similarities_matrix


Okey the first thing I want to test out is how long the training for different k values takes

In [25]:
knn_matrix, similarities_matrix = k_nearest_neighbors(train_data_matrix_0_fill, 3, 'euclidean')

Processed user 1/671 in 0.07 seconds.
Processed user 101/671 in 0.09 seconds.
Processed user 201/671 in 0.07 seconds.
Processed user 301/671 in 0.09 seconds.
Processed user 401/671 in 0.06 seconds.
Processed user 501/671 in 0.10 seconds.
Processed user 601/671 in 0.06 seconds.
Total time taken: 49.47 seconds.


## **4. Rule of Thumb:**
Some $K = \sqrt{n}$, where $n$ is the total number of samples in your training dataset. This formul can give you a starting point, but it's still essential to experiment around this value to find the optimal $K$.

In [26]:
train_data_matrix_avg_fill.shape

(671, 9066)

In [27]:
import math
Ksqrt = math.sqrt(train_data_matrix_avg_fill.shape[0])
Ksqrt

25.903667693977237

for now we're just gonna test out the time taken to compute the largest k's we are gonna use in crossvalidation to see if we need to implkament parlaaization in the case the algorithem takes too long

In [28]:
knn_matrix, similarities_matrix = k_nearest_neighbors(train_data_matrix_0_fill, int(2 * Ksqrt), 'euclidean')

Processed user 1/671 in 0.12 seconds.
Processed user 101/671 in 0.08 seconds.
Processed user 201/671 in 0.08 seconds.
Processed user 301/671 in 0.07 seconds.
Processed user 401/671 in 0.07 seconds.
Processed user 501/671 in 0.12 seconds.
Processed user 601/671 in 0.07 seconds.
Total time taken: 55.08 seconds.


In [29]:
knn_matrix.shape

(671, 51)

It really didn't take so long... so I don't think I'm gonna go after paralarization just yet.

## Implamenting Prediction and Evaluation

In [30]:
knn_matrix.shape # 671 user and its 51 neighbors

(671, 51)

In [31]:
train_data_matrix_0_fill.shape # 671 users and 9066 movies

(671, 9066)

In [32]:
train_data_matrix_0_fill

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [33]:
# set(train_data[['item_id']].sort_values(by=['item_id'])['item_id'].astype(int).to_list())

Below is the code for the function that predicts movie ratings for users

In [34]:
def predict_rating(user_index, movie_index, knn_matrix, similarities_matrix, ratings_matrix, use_similarities_matrix=True):
    # Extract the K nearest neighbors for the given user
    user_index = int(user_index)
    prev_index = 0
    movie_index = int(movie_index)
    neighbors_indices = knn_matrix[user_index]

    # Collect ratings for the target movie given by the neighbors
    if movie_index > ratings_matrix.shape[1]:
       movie_index = prev_index + 1

    neighbor_ratings = ratings_matrix[neighbors_indices, movie_index]

    # Also, collect the similarities for weighting, if applicable
    neighbor_similarities = similarities_matrix[user_index]

    # Filter out zeros (assuming they represent missing ratings)
    valid_indices = neighbor_ratings != 0
    valid_ratings = neighbor_ratings[valid_indices]
    valid_similarities = neighbor_similarities[valid_indices]

    prev_index = movie_index

    if use_similarities_matrix and valid_ratings.size > 0:
        # Compute weighted average of valid ratings
        weighted_sum = np.sum(valid_ratings * valid_similarities)
        sum_of_weights = np.sum(valid_similarities)
        predicted_rating = weighted_sum / sum_of_weights if sum_of_weights > 0 else np.nan
    elif valid_ratings.size > 0:
        # Compute simple average of valid ratings
        predicted_rating = valid_ratings.mean()
    else:
        # If there are no valid ratings, use the global average rating
        global_average = np.nanmean(ratings_matrix[ratings_matrix != 0])
        predicted_rating = global_average

    return predicted_rating


Below are a bunch of outputs for insights into how our matrecies and data are structured after training

In [35]:
knn_matrix[0, :]

array([546, 563,  72,  14, 623,  29, 451, 379, 293, 471, 467, 387, 231,
       101, 508,  22, 653, 579, 517, 310, 211, 118, 586,  55, 480, 241,
       305, 528, 663,  47, 357, 212, 298, 574, 595, 479, 606, 460, 198,
       104,  18, 474, 404, 406,  16, 462, 267, 261, 576, 543, 584])

In [36]:
train_data_matrix_0_fill[knn_matrix[0], 0]

array([3.5, 4. , 5. , 2. , 5. , 4. , 3.5, 4. , 4. , 5. , 4. , 0. , 3. ,
       0. , 3. , 3. , 5. , 4. , 5. , 3. , 3. , 2. , 0. , 4. , 4. , 5. ,
       3. , 4. , 3.5, 4. , 5. , 3. , 0. , 3. , 3.5, 5. , 4.5, 3.5, 0. ,
       0. , 3. , 4.5, 0. , 4. , 0. , 3. , 5. , 2.5, 5. , 0. , 4. ])

In [37]:
train_data

,user_id,item_id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
77406,69,344,4.0,1366831253
77407,452,4874,3.0,1044681111
77408,575,1201,5.0,1012594455
77409,17,1212,5.0,1127468800


In [38]:
train_data[(train_data.user_id == 1) & (train_data.item_id == 31)]

,user_id,item_id,rating,timestamp
0,1,31,2.5,1260759144


In [39]:
len(train_data)

77411

In [40]:
# for index, row in train_data.iterrows():
#         user_index = row['user_id']
#         movie_index1=row['item_id']
#         print(movie_index1)

In [41]:
# for index, row in train_data.iterrows():
#     print(row['rating'])

Below is the rather time expensive evaluate function that evaluates the perforamnce of the current knn model on a givven dataset. We will parallize this function shortly to make it run faster in some upcomming tests

In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

def evaluate_model(knn_matrix, similarities_matrix, ratings_matrix, dataset, use_similarities_matrix = True, print_ = True, print_every_n = 500):

    predictions = []
    actual_ratings = []

    # Iterate through each row in the dataset to make predictions
    i=0
    for index, row in dataset.iterrows():
        user_index = int(row['user_id'])-1
        movie_index = int(row['item_id'])-1
        actual_rating = row['rating']
        # Adjust indices for zero-based indexing
        if not np.isnan(actual_rating):

            predicted_rating = predict_rating(user_index, movie_index, knn_matrix, similarities_matrix, ratings_matrix, use_similarities_matrix)
            # print("evaluating on value", actual_rating)
            # print("predictet value", predicted_rating)

            predictions.append(predicted_rating)
            actual_ratings.append(actual_rating)
        else:
            # print("value should be nan: ", col)
            pass
        i+=1
        if(print_ and i%print_every_n==0): print(f"done with row index: {i}, out of: {len(dataset)}")

    # Calculate RMSE
    rmse = sqrt(mean_squared_error(actual_ratings, predictions))
    mae = mean_absolute_error(actual_ratings, predictions)

    if print_:
        print(f"RMSE: {rmse}")
        print(f"MAE: {mae}")

    return rmse, mae


Testing the predict function

In [43]:
predict_rating(0, 0, knn_matrix, similarities_matrix, train_data_matrix_0_fill)

3.8332888196358414

As we can see from above our knn predicted a rating of 3.83 for the user 0 and movie 0 and below we have some more insights into our data

In [44]:
dataset_array = train_data[['user_id', 'item_id', 'rating']].to_numpy()
dataset_array.shape

(77411, 3)

In [45]:
print(train_data_matrix_0_fill.shape)
print(knn_matrix.shape)

(671, 9066)
(671, 51)


### Now let's finally try evaluating our knn model

In [46]:
train_rmse, train_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_0_fill, train_data)
test_rmse, test_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_0_fill, test_data)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print('=' * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

done with row index: 500, out of: 77411
done with row index: 1000, out of: 77411
done with row index: 1500, out of: 77411
done with row index: 2000, out of: 77411
done with row index: 2500, out of: 77411
done with row index: 3000, out of: 77411
done with row index: 3500, out of: 77411
done with row index: 4000, out of: 77411
done with row index: 4500, out of: 77411
done with row index: 5000, out of: 77411
done with row index: 5500, out of: 77411
done with row index: 6000, out of: 77411
done with row index: 6500, out of: 77411
done with row index: 7000, out of: 77411
done with row index: 7500, out of: 77411
done with row index: 8000, out of: 77411
done with row index: 8500, out of: 77411
done with row index: 9000, out of: 77411
done with row index: 9500, out of: 77411
done with row index: 10000, out of: 77411
done with row index: 10500, out of: 77411
done with row index: 11000, out of: 77411
done with row index: 11500, out of: 77411
done with row index: 12000, out of: 77411
done with ro

================================

Training RMSE: 1.3108213065825864

Training MAE: 1.0372405987684707

================================

Test RMSE: 1.303092362800314

Test MAE: 1.0299218571321986

================================

These results aren't half bad for the little training it took compared to SVD. Interestingly it's also worth noting how similar the training and test RMSE errors are which is either an indication that I did something wrong in my knn implamentationor it shows how prone to overfiting on training data other algorithems like neural network based aproaches and or SVD are!

Now lets see what happens if we don't use the use_similarities_matrix = True and set it to False. These should be a more standard "and simple" KNN implamentation.

In [47]:
train_rmse, train_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_0_fill, train_data, False)
test_rmse, test_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_0_fill, test_data, False)

print('=' * 40)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print('=' * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

done with row index: 500, out of: 77411
done with row index: 1000, out of: 77411
done with row index: 1500, out of: 77411
done with row index: 2000, out of: 77411
done with row index: 2500, out of: 77411
done with row index: 3000, out of: 77411
done with row index: 3500, out of: 77411
done with row index: 4000, out of: 77411
done with row index: 4500, out of: 77411
done with row index: 5000, out of: 77411
done with row index: 5500, out of: 77411
done with row index: 6000, out of: 77411
done with row index: 6500, out of: 77411
done with row index: 7000, out of: 77411
done with row index: 7500, out of: 77411
done with row index: 8000, out of: 77411
done with row index: 8500, out of: 77411
done with row index: 9000, out of: 77411
done with row index: 9500, out of: 77411
done with row index: 10000, out of: 77411
done with row index: 10500, out of: 77411
done with row index: 11000, out of: 77411
done with row index: 11500, out of: 77411
done with row index: 12000, out of: 77411
done with ro

========================================

Training RMSE: 1.3117922938674418

Training MAE: 1.038320313088493

========================================

Test RMSE: 1.3041755659637155

Test MAE: 1.0309168630279977

========================================

A 0.0002 difference in RMSE! I guess it's noteable if you're operating on billions! I'm kidding it might just be something wrong with my implamentation or my k value or the structure of the 100k movies dataset.

Next I want to try what happens if we use the initialization of avrage values for missing values instead of a 0.

*I know we prepared the matrix before and I even explicitly mentioned it but let's do it again for the sake of cohearence*

In [48]:
# Create the pivot table
train_data_matrix = train_data.pivot_table(index='user_id', columns='item_id', values='rating')
# Fill missing values with user averages
train_data_matrix_avg_fill = train_data_matrix.apply(lambda x: x.fillna(user_rating_avg[x.name]), axis=1)
display(train_data_matrix_avg_fill.head())
train_data_matrix_avg_fill = train_data_matrix_avg_fill.to_numpy()
# Display the shape and first few rows of the matrix
print(train_data_matrix_avg_fill.shape)

item_id,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,...,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000
2,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,4.000000,...,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333
3,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,...,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000
4,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.000000,...,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259
5,3.917526,3.917526,4.000000,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,...,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526


(671, 9066)


In [49]:
knn_matrix, similarities_matrix = k_nearest_neighbors(train_data_matrix_avg_fill, int(2 * Ksqrt), 'euclidean')

Processed user 1/671 in 0.04 seconds.
Processed user 101/671 in 0.03 seconds.
Processed user 201/671 in 0.05 seconds.
Processed user 301/671 in 0.03 seconds.
Processed user 401/671 in 0.03 seconds.
Processed user 501/671 in 0.03 seconds.
Processed user 601/671 in 0.06 seconds.
Total time taken: 24.09 seconds.


In [50]:
train_rmse, train_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_avg_fill, train_data)
test_rmse, test_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_avg_fill, test_data)

print('KNN fill avg')
print('=' * 40)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print('=' * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

done with row index: 500, out of: 77411
done with row index: 1000, out of: 77411
done with row index: 1500, out of: 77411
done with row index: 2000, out of: 77411
done with row index: 2500, out of: 77411
done with row index: 3000, out of: 77411
done with row index: 3500, out of: 77411
done with row index: 4000, out of: 77411
done with row index: 4500, out of: 77411
done with row index: 5000, out of: 77411
done with row index: 5500, out of: 77411
done with row index: 6000, out of: 77411
done with row index: 6500, out of: 77411
done with row index: 7000, out of: 77411
done with row index: 7500, out of: 77411
done with row index: 8000, out of: 77411
done with row index: 8500, out of: 77411
done with row index: 9000, out of: 77411
done with row index: 9500, out of: 77411
done with row index: 10000, out of: 77411
done with row index: 10500, out of: 77411
done with row index: 11000, out of: 77411
done with row index: 11500, out of: 77411
done with row index: 12000, out of: 77411
done with ro

Wow okey! I was not expacting a bigger error by 0.4. My theory is  that an avrage initialization makes the data steer towards the avrage for no real practical reason and thereby maybe it takes different nearest neighbours. Or you know.. my implamentation is wrong.

## Testing different K values

In [51]:
knn_matrix_100, similarities_matrix_100 = k_nearest_neighbors(train_data_matrix_avg_fill, 100, 'euclidean')
train_rmse, train_mae = evaluate_model(knn_matrix_100, similarities_matrix_100, train_data_matrix_avg_fill, train_data)
test_rmse, test_mae = evaluate_model(knn_matrix_100, similarities_matrix_100, train_data_matrix_avg_fill, test_data)

print('KNN fill avg')
print('=' * 40)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print('=' * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

Processed user 1/671 in 0.09 seconds.
Processed user 101/671 in 0.05 seconds.
Processed user 201/671 in 0.05 seconds.
Processed user 301/671 in 0.05 seconds.
Processed user 401/671 in 0.05 seconds.
Processed user 501/671 in 0.08 seconds.
Processed user 601/671 in 0.05 seconds.
Total time taken: 35.78 seconds.
done with row index: 500, out of: 77411
done with row index: 1000, out of: 77411
done with row index: 1500, out of: 77411
done with row index: 2000, out of: 77411
done with row index: 2500, out of: 77411
done with row index: 3000, out of: 77411
done with row index: 3500, out of: 77411
done with row index: 4000, out of: 77411
done with row index: 4500, out of: 77411
done with row index: 5000, out of: 77411
done with row index: 5500, out of: 77411
done with row index: 6000, out of: 77411
done with row index: 6500, out of: 77411
done with row index: 7000, out of: 77411
done with row index: 7500, out of: 77411
done with row index: 8000, out of: 77411
done with row index: 8500, out of:

In [52]:

from concurrent.futures import ThreadPoolExecutor
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
import time


Bellow is the parallized code you don't need to put much thouight to it it's there mainly for speeding up my testing times.

In [53]:
def evaluate_model_concurrent(knn_matrix, similarities_matrix, ratings_matrix, dataset, use_similarities_matrix=True, workers=4):
    def worker(row):
        user_index, movie_index, actual_rating = row['user_id'].astype(int)-1, row['item_id'].astype(int)-1, row['rating']
        if not np.isnan(actual_rating):
            predicted_rating = predict_rating(user_index, movie_index, knn_matrix, similarities_matrix, ratings_matrix, use_similarities_matrix)
            return predicted_rating, actual_rating

    with ThreadPoolExecutor(max_workers=workers) as executor:
        results = executor.map(worker, [row for i, row in dataset.iterrows()])

    predictions, actual_ratings = zip(*[r for r in results if r is not None])

    # Calculate RMSE
    rmse = sqrt(mean_squared_error(actual_ratings, predictions))

    return rmse


In [54]:
import os
import multiprocessing

default_workers = os.cpu_count()
default_workers


2

Now let's get to the juicy part. Again like with the SVD testing let's write a function that tests different k values.
In this function we don't calculate a new knn model for each k because the first k-1 values in any knn row are the same as the the whole k-1 row

In [55]:
def test_different_ks(start_k, end_k, step, dataset_matrix, similarity_type='euclidean', use_similarities_matrix=True, train_data_portion=0.5, test_data_portion =1):
    train_rmse_values = []
    test_rmse_values = []

    cur_train_data = train_data[:int(len(train_data)*train_data_portion)]
    cur_test_data = train_data[:int(len(test_data)*test_data_portion)]

    # Calculate KNN and similarities matrices for the largest k value
    print(f"Calculating KNN and similarities matrices for k={end_k}")
    full_knn_matrix, full_similarities_matrix = k_nearest_neighbors(dataset_matrix, end_k, similarity_type)

    for k in range(start_k, end_k + 1, step):
        print(f"Testing with k={k}")
        start_time = time.time()

        # Use only the first k columns of the precomputed matrices
        cur_knn_matrix = full_knn_matrix[:, :k]
        cur_similarities_matrix = full_similarities_matrix[:, :k]

        train_rmse = evaluate_model(cur_knn_matrix, cur_similarities_matrix, dataset_matrix, cur_train_data, use_similarities_matrix)
        test_rmse = evaluate_model(cur_knn_matrix, cur_similarities_matrix, dataset_matrix, cur_test_data, use_similarities_matrix)

        train_rmse_values.append(train_rmse)
        test_rmse_values.append(test_rmse)
        end_time = time.time()
        print(f"k = {k}, train_rmse = {train_rmse}, test_rmse = {test_rmse}, time taken = {end_time - start_time:.2f} seconds")

    return full_knn_matrix, full_similarities_matrix, train_rmse_values, test_rmse_values


the parallel version of the above code that just calls the parallel evaluate function

In [56]:
def test_different_ks_parallel(start_k, end_k, step, dataset_matrix, similarity_type='euclidean', use_similarities_matrix=True, train_data_portion=0.5, test_data_portion =1, full_knn_matrix = None, full_similarities_matrix = None):
    cur_train_data = train_data[:int(len(train_data)*train_data_portion)]
    cur_test_data = train_data[:int(len(test_data)*test_data_portion)]
    train_rmse_values = []
    test_rmse_values = []

    #if(full_knn_matrix == None or full_similarities_matrix == None):
       # print(f"Calculating KNN and similarities matrices for k={end_k}")
        #full_knn_matrix, full_similarities_matrix = k_nearest_neighbors(dataset_matrix, end_k, similarity_type)

    for k in range(start_k, end_k + 1, step):
        print(f"Testing with k={k}")
        start_time = time.time()
        # Use only the first k columns of the precomputed matrices
        cur_knn_matrix = full_knn_matrix[:, :k]
        cur_similarities_matrix = full_similarities_matrix[:, :k]

        train_rmse = evaluate_model_concurrent(cur_knn_matrix, cur_similarities_matrix, dataset_matrix, cur_train_data, use_similarities_matrix,workers=16)
        test_rmse = evaluate_model_concurrent(cur_knn_matrix, cur_similarities_matrix, dataset_matrix, cur_test_data, use_similarities_matrix,workers=16)

        train_rmse_values.append(train_rmse)
        test_rmse_values.append(test_rmse)
        end_time = time.time()
        print(f"k = {k}, train_rmse = {train_rmse}, test_rmse = {test_rmse}")
        print(f"time taken to test k={k} - {end_time - start_time:.2f} seconds")

    return full_knn_matrix, full_similarities_matrix, train_rmse_values, test_rmse_values

In [57]:
knn_matrix, similarities_matrix, train_rmse_values, test_rmse_values = test_different_ks(
    start_k=0,
    end_k=21,
    step=1,
    dataset_matrix=train_data_matrix_0_fill,
    similarity_type='euclidean',
    use_similarities_matrix=True
)

In [58]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              int(4 * Ksqrt)

103

In [60]:
# import matplotlib.pyplot as plt

# # Assuming you have train_rmse_values and test_rmse_values lists
# ks = range(0, 21, 1)

# plt.plot(ks, train_rmse_values[0], label='Train RMSE', marker='o')
# plt.plot(ks, test_rmse_values[0], label='Test RMSE', marker='s')
# plt.xlabel('k')
# plt.ylabel('RMSE')
# plt.title('RMSE for Different Values of k')
# plt.legend()
# plt.grid(True)
# plt.show()

## Adding new Users or Items or rating and adressing the cold start problem

The cold start problem is a problem when adding a new user we don't know how to intiilize his user matrix.

Let's say we don't have the last user in our user matrix train_data_matrix_0_fill but we have his ratings. This simulates the scenario where we have a new user and we asked him to rate some movies to get a profile on him.

In [63]:
train_data_matrix_0_fill[-1]

array([5., 0., 0., ..., 0., 0., 0.])

In [64]:
new_user = train_data_matrix_0_fill[-1]
train_data_matrix_0_fill_1_missing = train_data_matrix_0_fill[:-1]
new_id = len(train_data_matrix_0_fill) - 1

In [65]:
knn_matrix_missing, similarities_matrix_missing = k_nearest_neighbors(train_data_matrix_0_fill_1_missing, 100)
knn_matrix_original, similarities_matrix_original = k_nearest_neighbors(train_data_matrix_0_fill, 100)

Processed user 1/670 in 0.35 seconds.
Processed user 101/670 in 0.09 seconds.
Processed user 201/670 in 0.09 seconds.
Processed user 301/670 in 0.31 seconds.
Processed user 401/670 in 0.16 seconds.
Processed user 501/670 in 0.09 seconds.
Processed user 601/670 in 0.09 seconds.
Total time taken: 71.94 seconds.
Processed user 1/671 in 0.15 seconds.
Processed user 101/671 in 0.09 seconds.
Processed user 201/671 in 0.09 seconds.
Processed user 301/671 in 0.10 seconds.
Processed user 401/671 in 0.15 seconds.
Processed user 501/671 in 0.09 seconds.
Processed user 601/671 in 0.09 seconds.
Total time taken: 69.82 seconds.


In [66]:
predict_rating(new_id, 0, knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill)

3.8313317530109186

In [67]:
predict_rating(new_id, 0, knn_matrix_missing, similarities_matrix_missing, train_data_matrix_0_fill_1_missing)

IndexError: index 670 is out of bounds for axis 0 with size 670

Above we have trained oour KNN matrix without the last user. Now let's try to add the last user to our systme so we need a function that adds a new user

As you can see with knn we can't really predict values for users that aren't in our system that is a downside of knn

Here is an optimized matrix I made that adds a new user to the end of our knn matrix in O(n) time and not O(n^2) so we can use this function on our dataset each time we get and add a new user in a theoretical example in practice

In [68]:
def add_new_user(dataset_matrix, knn_matrix, similarities_matrix, new_user, similarity_type='euclidean'):
    k = knn_matrix.shape[1]

    num_users = dataset_matrix.shape[0]
    new_best_similarities = np.full((k, 2), np.inf)
    for i in range(num_users):
        similarity = calculate_similarity_between_users(dataset_matrix[i], new_user, similarity_type)
        best_similarities = similarities_matrix[i]
        # Check if this similarity is better than the current stored similarities
        for l in range(k-1, -1, -1):
            if similarity < best_similarities[l]:
                if l < k - 1:
                    # Shift the later values down one position
                    best_similarities[l+1] = best_similarities[l]
                best_similarities[l] = similarity
            else:
                break
        for l in range(k-1, -1, -1):
            if similarity < new_best_similarities[l, 0]:
                if l < k - 1:
                    # Shift the later values down one position
                    new_best_similarities[l+1] = new_best_similarities[l]
                new_best_similarities[l] = [similarity, i]
            else:
                break
        similarities_matrix[i] = best_similarities

    # Append the new user to dataset_matrix


    # Prepare the arrays for the new user's nearest neighbors and their similarities
    new_user_neighbors_indices = new_best_similarities[:, 1].astype(int)
    new_user_similarities = new_best_similarities[:, 0]
    # Append the new user's nearest neighbors, their similarities and new_user to the dataset_matrix
    knn_matrix = np.vstack([knn_matrix, new_user_neighbors_indices])
    similarities_matrix = np.vstack([similarities_matrix, new_user_similarities])
    dataset_matrix = np.vstack([dataset_matrix, new_user])

    return knn_matrix, similarities_matrix, dataset_matrix

In [69]:
new_user

array([5., 0., 0., ..., 0., 0., 0.])

In [70]:
updated_knn_matrix, updated_similarities_matrix, updated_matrix_0_fill = add_new_user(
    train_data_matrix_0_fill_1_missing,
    knn_matrix_missing,
    similarities_matrix_missing,
    new_user
    )

In [71]:
predict_rating(new_id, 0, knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill)

3.8313317530109186

In [72]:
predict_rating(new_id, 0, updated_knn_matrix, updated_similarities_matrix, updated_matrix_0_fill)

3.8313317530109186

As you can see above now that we added the user into our "system" we can predict with the new user

Let's test if our system that adds a new user has the same accuracy as the original if my implamentation is correct they should be the same

In [73]:
train_rmse, train_mae = evaluate_model(knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill, train_data, print_=False)
test_rmse, test_mae = evaluate_model(knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill, test_data, print_=False)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print("=" * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

Training RMSE: 1.323584415100453
Training MAE: 1.0450259504588382
Test RMSE: 1.317197583268473
Test MAE: 1.039886370711994


In [75]:
train_rmse, train_mae = evaluate_model(updated_knn_matrix, updated_similarities_matrix, updated_matrix_0_fill, train_data, print_=False)
test_rmse, test_mae = evaluate_model(updated_knn_matrix, updated_similarities_matrix, updated_matrix_0_fill, test_data, print_=False)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print("=" * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

Training RMSE: 1.323584415100453
Training MAE: 1.0450259504588382
Test RMSE: 1.317197583268473
Test MAE: 1.039886370711994


Now we are going to simulate the scenario where we get a new item so add a new movie. TO do this we first need to add an empty movie with 0 ratings. And then we add individual ratings as entries. This is bnecause in a real scenario when we add a new movie we have 0 ratings for it because no one has rated it yet. Then we can ask users some  what they think of it and add those ratings individualy  

In [76]:
# Simulate removing a movie column
new_movie_index = len(train_data_matrix_0_fill[0])-1
new_movie = train_data_matrix_0_fill[:, -1]  # Save the last column to a variable
train_data_matrix_0_fill_less_one_movie = np.delete(train_data_matrix_0_fill, -1, axis=1)  # Remove the last column

# Simulate adding a new movie item by adding a column of zeros to the end of the original array
# Note: You'll want to ensure the number of rows matches for the zeros column you're adding
num_users = train_data_matrix_0_fill_less_one_movie.shape[0]


In [77]:
knn_matrix_missing_movie, similarities_matrix_missing_movie = k_nearest_neighbors(train_data_matrix_0_fill_less_one_movie, 100)
knn_matrix_original, similarities_matrix_original = k_nearest_neighbors(train_data_matrix_0_fill, 100)

Processed user 1/671 in 0.15 seconds.
Processed user 101/671 in 0.19 seconds.
Processed user 201/671 in 0.21 seconds.
Processed user 301/671 in 0.14 seconds.
Processed user 401/671 in 0.14 seconds.
Processed user 501/671 in 0.14 seconds.
Processed user 601/671 in 0.15 seconds.
Total time taken: 106.60 seconds.
Processed user 1/671 in 0.09 seconds.
Processed user 101/671 in 0.10 seconds.
Processed user 201/671 in 0.14 seconds.
Processed user 301/671 in 0.09 seconds.
Processed user 401/671 in 0.09 seconds.
Processed user 501/671 in 0.09 seconds.
Processed user 601/671 in 0.09 seconds.
Total time taken: 67.53 seconds.


Now we want to split up the new_movie column into individual ratings to simulate each user rating the new movce when asked what they think about it. (This can also be thought as simulating new user ratings for any movie not necasaraly the new one added)

In [78]:
individual_ratings = []

# Iterate through the new_movie column to find non-zero ratings
for user_index, rating in enumerate(new_movie):
    if rating != 0:
        # Append the user index and rating as a sublist
        individual_ratings.append([user_index, new_movie_index, rating])

print(individual_ratings)
new_rating = individual_ratings[0]
print(new_rating)

[[546, 9065, 5.0]]
[546, 9065, 5.0]


First let's see what happens when we try to predict.

In [79]:
predict_rating(new_rating[0], new_rating[1], knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill)

3.5327343659169888

In [80]:
predict_rating(new_rating[0], new_rating[1], knn_matrix_missing_movie, similarities_matrix_missing_movie, train_data_matrix_0_fill_less_one_movie)

IndexError: index 9065 is out of bounds for axis 1 with size 9065

Now we need to add this rating to our Knn system similaraly to how we added a new user but just for a single rating this should also take O(n) time

In [81]:
def alter_new_rating(dataset_matrix, knn_matrix, similarities_matrix, new_movie_ratring, similarity_type='euclidean'):
    rating_user_index = new_movie_ratring[0]
    rating_movie_index = new_movie_ratring[1]
    rating = new_movie_ratring[2]
    k = knn_matrix.shape[1]
    altering_user = dataset_matrix[rating_user_index]
    altering_user[rating_movie_index] = rating
    num_users = dataset_matrix.shape[0]

    new_best_similarities = np.full((k, 2), np.inf)
    for i in range(num_users):

        if i == rating_user_index: continue

        similarity = calculate_similarity_between_users(dataset_matrix[i], altering_user, similarity_type)
        best_similarities = similarities_matrix[i]
        # Check if this similarity is better than the current stored similarities
        for l in range(k-1, -1, -1):
            if similarity < best_similarities[l]:
                if l < k - 1:
                    # Shift the later values down one position
                    best_similarities[l+1] = best_similarities[l]
                best_similarities[l] = similarity
            else:
                break
        for l in range(k-1, -1, -1):
            if similarity < new_best_similarities[l, 0]:
                if l < k - 1:
                    # Shift the later values down one position
                    new_best_similarities[l+1] = new_best_similarities[l]
                new_best_similarities[l] = [similarity, i]
            else:
                break
        similarities_matrix[i] = best_similarities


    # Prepare the arrays for the new user's nearest neighbors and their similarities
    new_user_neighbors_indices = new_best_similarities[:, 1].astype(int)
    new_user_similarities = new_best_similarities[:, 0]
    # change the new user's nearest neighbors, their similarities and new_user to the dataset_matrix
    knn_matrix[rating_user_index] = new_user_neighbors_indices
    similarities_matrix[rating_user_index] = new_user_similarities

    dataset_matrix[rating_user_index] = altering_user

    return knn_matrix, similarities_matrix, dataset_matrix



In [82]:
new_rating

[546, 9065, 5.0]

First we add an empty column to simulate a new movie being addedwithout any ratings:

In [83]:
zeros_column = np.zeros((num_users, 1))
train_data_matrix_0_fill_with_new_movie = np.hstack([train_data_matrix_0_fill_less_one_movie, zeros_column])

In [84]:
len(train_data_matrix_0_fill_less_one_movie[0])

9065

In [85]:
len(train_data_matrix_0_fill_with_new_movie[0])

9066

Now we add a rating a user made for the new movie

In [86]:
updated_knn_matrix_movie, updated_similarities_matrix_movie, updated_matrix_0_fill_movie = alter_new_rating(
    train_data_matrix_0_fill_with_new_movie,
    knn_matrix_missing_movie,
    similarities_matrix_missing_movie,
    new_rating
    )

In [87]:
predict_rating(new_rating[0], new_rating[1], knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill)

3.5327343659169888

In [88]:
predict_rating(new_rating[0], new_rating[1], updated_knn_matrix_movie, updated_similarities_matrix_movie, updated_matrix_0_fill_movie)

3.5327343659169888

As we can see we we're able to implament adding a user and movie in O(n) time
Also by using the alter_new_rating function we can simualte any user making a new rating and updating the system in O(n) time.
But what if we could update a new rating in O(1)

We can use a partial update and update just the knn values of the user who made the new rating so his predictions are on point but others aren't so we treat the others like the new user never made a new rating. Then we can update the whole system in O(n^2) time.

This aproach is usefull when we have many new ratings by many users which is the case in real life examples so we update each user in O(1) time and then when we decide durring a low trafic time for our system we update the whole system in O(n^2)  time . (During the night  for example)

In [89]:
import numpy as np
import random

# Assuming train_data_matrix_0_fill_less_one_movie is your numpy matrix
matrix = train_data_matrix_0_fill

# Number of random non-zero ratings to select
n = 10  # for example

# Find all non-zero ratings along with their indices
non_zero_indices = np.argwhere(matrix != 0)

# Randomly select n entries from non_zero_indices
random_selections = random.sample(list(non_zero_indices), n)

# Create the list of [user_index, movie_index, rating] for the random selections
random_ratings = [[user_index, movie_index, matrix[user_index, movie_index]] for user_index, movie_index in random_selections]

print(random_ratings)

[[240, 196, 3.0], [211, 6278, 4.0], [606, 675, 4.0], [12, 2894, 3.0], [563, 1647, 4.0], [8, 1698, 4.0], [462, 48, 5.0], [620, 3668, 4.0], [511, 3845, 4.5], [194, 3047, 2.0]]


In [90]:
missing_values_matrix_0_fill = train_data_matrix_0_fill.copy()

for rating in random_ratings:
    missing_values_matrix_0_fill[rating[0], rating[1]] = 0

In [91]:
knn_matrix_missing_values, similarities_matrix_missing_values = k_nearest_neighbors(missing_values_matrix_0_fill, 100)
knn_matrix_original, similarities_matrix_original = k_nearest_neighbors(train_data_matrix_0_fill, 100)

Processed user 1/671 in 0.10 seconds.
Processed user 101/671 in 0.05 seconds.
Processed user 201/671 in 0.05 seconds.
Processed user 301/671 in 0.10 seconds.
Processed user 401/671 in 0.53 seconds.
Processed user 501/671 in 0.05 seconds.
Processed user 601/671 in 0.06 seconds.
Total time taken: 49.00 seconds.
Processed user 1/671 in 0.10 seconds.
Processed user 101/671 in 0.16 seconds.
Processed user 201/671 in 0.15 seconds.
Processed user 301/671 in 0.14 seconds.
Processed user 401/671 in 0.09 seconds.
Processed user 501/671 in 0.10 seconds.
Processed user 601/671 in 0.09 seconds.
Total time taken: 84.65 seconds.


In [92]:
for rating in random_ratings:
    print(predict_rating(rating[0], rating[1], knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill))

3.9963621189419256
4.067963734617265
3.42508275240639
3.290344472938223
3.2792638437412003
3.529297294017251
4.220275335118865
3.6121916403430756
3.9827139032008003
3.609358190798711


In [93]:
for rating in random_ratings:
    print(predict_rating(rating[0], rating[1], knn_matrix_missing_values, similarities_matrix_missing_values, missing_values_matrix_0_fill))

3.995635517146827
4.067816431825159
3.424782384726903
3.231232461313176
3.2790903355967993
3.5287980605182034
4.2373994911731225
3.612192958029144
3.954884812871316
3.609277830085633


We now write the function that will only alter the knn and similarety values for the altering user so he has the correct recomendations but other users are the same as if nothing changed

In [94]:
def partly_alter_new_rating(dataset_matrix, knn_matrix, similarities_matrix, new_movie_ratring, similarity_type='euclidean'):
    rating_user_index = new_movie_ratring[0]
    rating_movie_index = new_movie_ratring[1]
    rating = new_movie_ratring[2]
    k = knn_matrix.shape[1]
    altering_user = dataset_matrix[rating_user_index]
    altering_user[rating_movie_index] = rating

    #new_best_similarities = similarities_matrix[rating_user_index]
    num_users = dataset_matrix.shape[0]

    new_best_similarities = np.full((k, 2), np.inf)
    for i in range(num_users):
        if i == rating_user_index: continue

        similarity = calculate_similarity_between_users(dataset_matrix[i], altering_user, similarity_type)

        for l in range(k-1, -1, -1):
            if similarity < new_best_similarities[l, 0]:
                if l < k - 1:
                    # Shift the later values down one position
                    new_best_similarities[l+1] = new_best_similarities[l]
                new_best_similarities[l] = [similarity, i]
            else:
                break


    # Prepare the arrays for the new user's nearest neighbors and their similarities
    new_user_neighbors_indices = new_best_similarities[:, 1].astype(int)
    new_user_similarities = new_best_similarities[:, 0]
    # change the new user's nearest neighbors, their similarities and new_user to the dataset_matrix
    knn_matrix[rating_user_index] = new_user_neighbors_indices
    similarities_matrix[rating_user_index] = new_user_similarities

    dataset_matrix[rating_user_index] = altering_user

    return knn_matrix, similarities_matrix, dataset_matrix



I think we could easily optimize the abvoe function further

We will now simulate many users doing this each without knowing about the other

In [95]:
combined_partly_updated_knn = knn_matrix_missing_values.copy()
combined_partly_updated_similarities = similarities_matrix_missing_values.copy()
combined_partly_updated_0_fill = missing_values_matrix_0_fill.copy()

all_partly_updated_knn = []
all_partly_updated_similarities = []
all_partly_updated_0_fill = []

for rating in random_ratings:
    updated_knn_matrix_movie, updated_similarities_matrix_movie, updated_matrix_0_fill_movie = partly_alter_new_rating(
    combined_partly_updated_0_fill,
    combined_partly_updated_knn,
    combined_partly_updated_similarities,
    rating
    )

    all_partly_updated_knn.append(updated_knn_matrix_movie)
    all_partly_updated_similarities.append(updated_similarities_matrix_movie)
    all_partly_updated_0_fill.append(updated_matrix_0_fill_movie)

    combined_partly_updated_knn = updated_knn_matrix_movie
    combined_partly_updated_similarities = updated_similarities_matrix_movie
    combined_partly_updated_0_fill = updated_matrix_0_fill_movie

In [96]:
for rating in random_ratings:
    print(predict_rating(rating[0], rating[1], knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill))

3.9963621189419256
4.067963734617265
3.42508275240639
3.290344472938223
3.2792638437412003
3.529297294017251
4.220275335118865
3.6121916403430756
3.9827139032008003
3.609358190798711


In [97]:
for rating in random_ratings:
    print(predict_rating(rating[0], rating[1], all_partly_updated_knn[0], all_partly_updated_similarities[0], all_partly_updated_0_fill[0]))

3.99629630553757
4.067963734617265
3.425146479414679
3.290344472938223
3.2792638437412003
3.529297294017251
4.220275335118865
3.6121916403430756
3.9827139032008003
3.609358190798711


In [98]:
for rating in random_ratings:
    print(predict_rating(rating[0], rating[1], combined_partly_updated_knn, combined_partly_updated_similarities, combined_partly_updated_0_fill))

3.99629630553757
4.067963734617265
3.425146479414679
3.290344472938223
3.2792638437412003
3.529297294017251
4.220275335118865
3.6121916403430756
3.9827139032008003
3.609358190798711


Now on a not so bussy up time we can update the whole matrix for all users

In [99]:
fully_updated_knn_matrix, fully_updated_similarities_matrix = k_nearest_neighbors(combined_partly_updated_0_fill, 100, 'euclidean')

Processed user 1/671 in 0.10 seconds.
Processed user 101/671 in 0.05 seconds.
Processed user 201/671 in 0.05 seconds.
Processed user 301/671 in 0.09 seconds.
Processed user 401/671 in 0.05 seconds.
Processed user 501/671 in 0.05 seconds.
Processed user 601/671 in 0.05 seconds.
Total time taken: 43.86 seconds.


In [100]:
train_rmse, train_mae = evaluate_model(knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill, train_data, print_=False)
test_rmse, test_mae = evaluate_model(knn_matrix_original, similarities_matrix_original, train_data_matrix_0_fill, test_data, print_=False)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print("=" * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

Training RMSE: 1.323584415100453
Training MAE: 1.0450259504588382
Test RMSE: 1.317197583268473
Test MAE: 1.039886370711994


In [101]:
train_rmse, train_mae = evaluate_model(combined_partly_updated_knn, combined_partly_updated_similarities, combined_partly_updated_0_fill, train_data, print_=False)
test_rmse, test_mae = evaluate_model(combined_partly_updated_knn, combined_partly_updated_similarities, combined_partly_updated_0_fill, test_data, print_=False)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print("=" * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

Training RMSE: 1.3235567119345608
Training MAE: 1.0449750995923466
Test RMSE: 1.3170976393628866
Test MAE: 1.0399033719938375


In [102]:
train_rmse, train_mae = evaluate_model(fully_updated_knn_matrix, fully_updated_similarities_matrix, combined_partly_updated_0_fill, train_data, print_=False)
test_rmse, test_mae = evaluate_model(fully_updated_knn_matrix, fully_updated_similarities_matrix, combined_partly_updated_0_fill, test_data, print_=False)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print("=" * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

Training RMSE: 1.323584415100453
Training MAE: 1.0450259504588382
Test RMSE: 1.317197583268473
Test MAE: 1.039886370711994
